THIS CODE PERFORM LABELLING AT ANCHOR LEVEL
THIS INSERT PHOSC REPRESENTATION AT ANCOR LEVEL

In [1]:
import os
import cv2
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf

#print("\n\t tf:",tf.__version__)
#tf.enable_eager_execution()
# from tensorflow.keras.utils import plot_model
#from yolov3.dataset import Dataset
#from yolov3.yolov4 import Create_Yolo, compute_loss,Create_YoloPhosc
#from yolov3.utils import load_yolo_weights
from yolov3.configs import *
#from evaluate_mAP import get_mAP

print("\n\t YOLO_TYPE:",YOLO_TYPE)




[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1794188377882962759
]

	 YOLO_TYPE: yolov3


In [2]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
import pickle
from yolov3.utils import read_class_names, image_preprocess
from yolov3.yolov3 import bbox_iou
from yolov3.configs import *


In [5]:
TRAIN_BATCH_SIZE
YOLO_STRIDES
classes = read_class_names(TRAIN_CLASSES)
#classes
anchors = (np.array(YOLO_ANCHORS).T/np.array(YOLO_STRIDES)).T
#np.array(YOLO_ANCHORS)
#np.array(YOLO_STRIDES)
#anchors
train_output_sizes=426//np.array(YOLO_STRIDES)
print(train_output_sizes)
'''
batch_label_sbbox = np.zeros((self.batch_size, self.train_output_sizes[0], self.train_output_sizes[0], self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)
batch_label_mbbox = np.zeros((self.batch_size, self.train_output_sizes[1], self.train_output_sizes[1], self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)
batch_label_lbbox = np.zeros((self.batch_size, self.train_output_sizes[2], self.train_output_sizes[2], self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)

'''

temp=np.zeros((1,int(train_output_sizes[2]), int(train_output_sizes[2]),3, 5 +1))
print(temp.shape)

temp[:,:,:,0,:]=-255
temp[:,:,:,1,:]=10
temp[:,:,:,2,:]=255

#sum(temp[:,:,:,2,0])

batch_mbboxes = np.zeros((1, YOLO_MAX_BBOX_PER_SCALE, 4), dtype=np.float32)
#batch_lbboxes = np.zeros((1, YOLO_MAX_BBOX_PER_SCALE, 4), dtype=np.float32)
print("\n\t batch_mbboxes:",batch_mbboxes.shape)


[53 26 13]
(1, 13, 13, 3, 6)

	 batch_mbboxes: (1, 100, 4)


In [6]:
class Dataset1(object):
    # Dataset preprocess implementation
    def __init__(self, dataset_type,TEST_INPUT_SIZE=TEST_INPUT_SIZE):
        
        f = open("./data/tempCord.json") 
        self.tempCord1= json.load(f)
                
        with open('filename.pickle', 'rb') as handle:
            self.tempText1 = pickle.load(handle)
            
        self.annot_path  = TRAIN_ANNOT_PATH if dataset_type == 'train' else TEST_ANNOT_PATH
        self.input_sizes = TRAIN_INPUT_SIZE if dataset_type == 'train' else TEST_INPUT_SIZE #416
        self.batch_size  = TRAIN_BATCH_SIZE if dataset_type == 'train' else TEST_BATCH_SIZE
        self.data_aug    = TRAIN_DATA_AUG   if dataset_type == 'train' else TEST_DATA_AUG

        self.train_yolo_tiny = TRAIN_YOLO_TINY
        self.train_input_sizes = TRAIN_INPUT_SIZE # 416 
        self.strides = np.array(YOLO_STRIDES) # [8,16,32]
        self.classes = read_class_names(TRAIN_CLASSES) 
        self.num_classes = len(self.classes)
        #print("\n\t self.num_classes=",self.num_classes)
        self.anchors = (np.array(YOLO_ANCHORS).T/self.strides).T
        self.anchor_per_scale = YOLO_ANCHOR_PER_SCALE
        self.max_bbox_per_scale = YOLO_MAX_BBOX_PER_SCALE

        self.annotations = self.load_annotations(dataset_type)
        #print("\n\t 2.no of coordinate after:",len(self.tempCord1))

        self.num_samples = len(self.annotations)
        self.num_batchs = int(np.ceil(self.num_samples / self.batch_size))
        #print("\n\t num_samples =",self.num_samples)
        #print("\n\t num_batchs =",self.num_batchs)
        self.batch_count = 0
        
    def load_annotations(self, dataset_type):
        final_annotations = []
        with open(self.annot_path, 'r') as f:
            txt = f.read().splitlines()
            annotations = [line.strip() for line in txt if len(line.strip().split()[1:]) != 0]

        #print(annotations)
        #np.random.shuffle(annotations)

        for annotation in annotations:
            # fully parse annotations
            line = annotation.split()
            image_path, index = "", 1

            #print("\n\t line:",line)
            for i, one_line in enumerate(line):
                if not one_line.replace(",","").isnumeric():
                    if image_path != "": image_path += " "
                    image_path += one_line
                else:
                    index = i
                    break

            image_path=line.pop(0)
            #print("\n\t path-->",os.path.exists(image_path))
            #print("\n\t path:",image_path)
            if not os.path.exists(image_path):
                raise KeyError("%s does not exist ... " %image_path)
            if TRAIN_LOAD_IMAGES_TO_RAM:
                image = cv2.imread(image_path)
            else:
                image = ''
            final_annotations.append([image_path, line[index:], image])
            
            '''
            for cord in line[index:]:
                #'[361, 587, 480, 652]'  2.cord: ['603', '603', '689', '652']

                #print("\n\t 1.cord:",cord)

                cord=cord.split(",")[:4]
                #print("\n\t 2.cord:",cord)
                
                cord = str([int(i) for i in cord])
                #print("***>",cord)
                txt=self.tempCord1[cord]

                #txt=self.tempCord1[str(list(cord))]
                #print("\n\t word:",txt)
                vec=self.tempText1[txt]
                #print("\n\t phoc:",len(vec['phoc']),"\t phos:",len(vec['phos']))
                #print("\n\t phoc:",vec['phoc'])
                #print("\n\t phoc:",vec['phos'])
            '''
                
            #print("\n\t record:",image_path,"\t ",image.shape,"\t index:",index,"\t le:",len(line[index:]))
            #print("\n\t line:",line[index:])
            
            #input("check!!!")
        return final_annotations

    def __iter__(self):
        return self

    def Delete_bad_annotation(self, bad_annotation):
        print(f'Deleting {bad_annotation} annotation line')
        bad_image_path = bad_annotation[0]
        bad_image_name = bad_annotation[0].split('/')[-1] # can be used to delete bad image
        bad_xml_path = bad_annotation[0][:-3]+'xml' # can be used to delete bad xml file

        # remove bad annotation line from annotation file
        with open(self.annot_path, "r+") as f:
            d = f.readlines()
            f.seek(0)
            for i in d:
                if bad_image_name not in i:
                    f.write(i)
            f.truncate()

    def __next__(self):
        with tf.device('/gpu:0'):
            self.train_input_size = random.choice([self.train_input_sizes])
            self.train_output_sizes = self.train_input_size // self.strides

            batch_image = np.zeros((self.batch_size, self.train_input_size, self.train_input_size, 3), dtype=np.float32)

            if self.train_yolo_tiny:
                batch_label_mbbox = np.zeros((self.batch_size, self.train_output_sizes[0], self.train_output_sizes[0], self.anchor_per_scale, 5 + self.num_classes+769), dtype=np.float32)
                batch_label_lbbox = np.zeros((self.batch_size, self.train_output_sizes[1], self.train_output_sizes[1], self.anchor_per_scale, 5 + self.num_classes+769), dtype=np.float32)
            else:
                batch_label_sbbox = np.zeros((self.batch_size, self.train_output_sizes[0], self.train_output_sizes[0], self.anchor_per_scale, 5 + self.num_classes+769), dtype=np.float32)
                batch_label_mbbox = np.zeros((self.batch_size, self.train_output_sizes[1], self.train_output_sizes[1], self.anchor_per_scale, 5 + self.num_classes+769), dtype=np.float32)
                batch_label_lbbox = np.zeros((self.batch_size, self.train_output_sizes[2], self.train_output_sizes[2], self.anchor_per_scale, 5 + self.num_classes+769), dtype=np.float32)

                batch_sbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4+769), dtype=np.float32)

            batch_mbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4+769), dtype=np.float32)
            batch_lbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4+769), dtype=np.float32)

            exceptions = False
            num = 0
            if self.batch_count < self.num_batchs:
                while num < self.batch_size:
                    index = self.batch_count * self.batch_size + num
                    if index >= self.num_samples: index -= self.num_samples
                    annotation = self.annotations[index]
                    
                    print("\n\t annotation name=",annotation[0])
                    #print("\n\t 1.annotation =",len(annotation[1]))
                    #print("\n\t annotation =",annotation[2].shape)

                    image, bboxes,tempCord11 = self.parse_annotation(annotation)
                    #print("\n\t image:",image.shape)
                    #print("\n\t bboxes shape =",bboxes.shape)
                    #print("\n\t 2.bboxes =",bboxes)
                    
                    try:
                        if self.train_yolo_tiny:
                            label_mbbox, label_lbbox, mbboxes, lbboxes = self.preprocess_true_boxes(bboxes,tempCord11)
                        else:
                            label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes = self.preprocess_true_boxes(bboxes,tempCord11)
                    except IndexError:
                        exceptions = True
                        self.Delete_bad_annotation(annotation)
                        print("IndexError, something wrong with", annotation[0], "removed this line from annotation file")
                  
                    #label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes
                    '''
                    print("\n\t label_sbbox:",label_sbbox.shape)
                    print("\n\t label_mbbox:",label_mbbox.shape)
                    print("\n\t label_lbbox:",label_lbbox.shape)
                    print("\n\t sbboxes:",sbboxes.shape)
                    print("\n\t mbboxes:",mbboxes.shape)
                    print("\n\t lbboxes:", lbboxes.shape)
                    '''
                    batch_image[num, :, :, :] = image
                    batch_label_mbbox[num, :, :, :, :] = label_mbbox
                    batch_label_lbbox[num, :, :, :, :] = label_lbbox
                    batch_mbboxes[num, :, :] = mbboxes
                    batch_lbboxes[num, :, :] = lbboxes
                    if not self.train_yolo_tiny:
                        batch_label_sbbox[num, :, :, :, :] = label_sbbox
                        batch_sbboxes[num, :, :] = sbboxes

                    num += 1

                if exceptions:
                    print('\n')
                    raise Exception("There were problems with dataset, I fixed them, now restart the training process.")
                self.batch_count += 1
                if not self.train_yolo_tiny:
                    batch_smaller_target = batch_label_sbbox, batch_sbboxes
                batch_medium_target  = batch_label_mbbox, batch_mbboxes
                batch_larger_target  = batch_label_lbbox, batch_lbboxes

                if self.train_yolo_tiny:
                    return batch_image, (batch_medium_target, batch_larger_target)
                return batch_image, (batch_smaller_target, batch_medium_target, batch_larger_target)
            else:
                self.batch_count = 0
                np.random.shuffle(self.annotations)
                raise StopIteration

    def random_horizontal_flip(self, image, bboxes):
        if random.random() < 0.5:
            _, w, _ = image.shape
            image = image[:, ::-1, :]
            bboxes[:, [0,2]] = w - bboxes[:, [2,0]]

        return image, bboxes

    def random_crop(self, image, bboxes):
        if random.random() < 0.5:
            h, w, _ = image.shape
            max_bbox = np.concatenate([np.min(bboxes[:, 0:2], axis=0), np.max(bboxes[:, 2:4], axis=0)], axis=-1)

            max_l_trans = max_bbox[0]
            max_u_trans = max_bbox[1]
            max_r_trans = w - max_bbox[2]
            max_d_trans = h - max_bbox[3]

            crop_xmin = max(0, int(max_bbox[0] - random.uniform(0, max_l_trans)))
            crop_ymin = max(0, int(max_bbox[1] - random.uniform(0, max_u_trans)))
            crop_xmax = max(w, int(max_bbox[2] + random.uniform(0, max_r_trans)))
            crop_ymax = max(h, int(max_bbox[3] + random.uniform(0, max_d_trans)))

            image = image[crop_ymin : crop_ymax, crop_xmin : crop_xmax]

            bboxes[:, [0, 2]] = bboxes[:, [0, 2]] - crop_xmin
            bboxes[:, [1, 3]] = bboxes[:, [1, 3]] - crop_ymin

        return image, bboxes

    def random_translate(self, image, bboxes):
        if random.random() < 0.5:
            h, w, _ = image.shape
            max_bbox = np.concatenate([np.min(bboxes[:, 0:2], axis=0), np.max(bboxes[:, 2:4], axis=0)], axis=-1)

            max_l_trans = max_bbox[0]
            max_u_trans = max_bbox[1]
            max_r_trans = w - max_bbox[2]
            max_d_trans = h - max_bbox[3]

            tx = random.uniform(-(max_l_trans - 1), (max_r_trans - 1))
            ty = random.uniform(-(max_u_trans - 1), (max_d_trans - 1))

            M = np.array([[1, 0, tx], [0, 1, ty]])
            image = cv2.warpAffine(image, M, (w, h))

            bboxes[:, [0, 2]] = bboxes[:, [0, 2]] + tx
            bboxes[:, [1, 3]] = bboxes[:, [1, 3]] + ty

        return image, bboxes

    def parse_annotation(self, annotation, mAP = 'False'):
        if TRAIN_LOAD_IMAGES_TO_RAM:
            image_path = annotation[0]
            image = annotation[2]
        else:
            image_path = annotation[0]
            image = cv2.imread(image_path)

        bboxes = np.array([list(map(int, box.split(','))) for box in annotation[1]])

        if self.data_aug:
            image, bboxes = self.random_horizontal_flip(np.copy(image), np.copy(bboxes))
            image, bboxes = self.random_crop(np.copy(image), np.copy(bboxes))
            image, bboxes = self.random_translate(np.copy(image), np.copy(bboxes))

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if mAP == True:
            return image, bboxes

        image, bboxes,tempCord11 = image_preprocess(np.copy(image), [self.input_sizes, self.input_sizes],self.tempCord1,self.tempText1, np.copy(bboxes))
        return image, bboxes,tempCord11

    def preprocess_true_boxes(self, bboxes,tempCord11):
        OUTPUT_LEVELS = len(self.strides)

        label = [np.zeros((self.train_output_sizes[i], self.train_output_sizes[i], self.anchor_per_scale,
                           5 + self.num_classes+769)) for i in range(OUTPUT_LEVELS)]# change
        bboxes_xywh = [np.zeros((self.max_bbox_per_scale, 4+769)) for _ in range(OUTPUT_LEVELS)]# change
        bbox_count = np.zeros((OUTPUT_LEVELS,))

        #print("\n\tbboxes_xywh:",len(bboxes_xywh),"\t shape:",bboxes_xywh[0].shape)
        # (3,100,4)
        for bbox in bboxes:
            
            #print("\n\t bbox=",bbox)
            orgBbox=bbox[:4]
            #print("\n\t 1.orgBbox:",orgBbox)#[80 20 83 21]-->'[80, 20, 83, 21]'
            #orgBbox=orgBbox[0]+orgBbox[1]+orgBbox[2]+orgBbox[3]
            key=str("[")+str(bbox[0])+", "+str(bbox[1])+", "+str(bbox[2])+", "+str(bbox[3])+"]"
            #print("\n\t key:",key)
            key=tempCord11[key]
            
            '''
            try:
                print("\n\t text string:",tempCord11[key])
            except Exception as e:
                print("\n\t exception key:",key)
            '''
                #input("exception!!!")
            #orgBbox=orgBbox.split(" ")
            #print("\n\t 2.orgBbox:",orgBbox)
            #orgBbox=orgBbox="".join(", ")
            #print("\n\t 3.orgBbox:",orgBbox)

            bbox_coor = bbox[:4]
            bbox_class_ind = bbox[4]

            onehot = np.zeros(self.num_classes, dtype=np.float)
            onehot[bbox_class_ind] = 1.0
            uniform_distribution = np.full(self.num_classes, 1.0 / self.num_classes)
            deta = 0.01
            smooth_onehot = onehot * (1 - deta) + deta * uniform_distribution
            
            bbox_xywh = np.concatenate([(bbox_coor[2:4] + bbox_coor[:2]) * 0.5, bbox_coor[2:4] - bbox_coor[:2]], axis=-1) # change
            #print("\n\t 0.bbox_xywh centre HW=",bbox_xywh)
            
            
            bbox_xywh_scaled = 1.0 * bbox_xywh[np.newaxis, :4] / self.strides[:, np.newaxis]
            #print("\n\t 1.bbox_xywh_scaled =",bbox_xywh_scaled)
            
            iou = []
            exist_positive = False
            for i in range(OUTPUT_LEVELS):#range(3):
                anchors_xywh = np.zeros((self.anchor_per_scale, 4))
                #print("\n\t anchors_xywh:",anchors_xywh) # (3,4)
                anchors_xywh[:, 0:2] = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32) + 0.5
                anchors_xywh[:, 2:4] = self.anchors[i]
                #print("\n\t anchors_xywh:",anchors_xywh)
                
                iou_scale = bbox_iou(bbox_xywh_scaled[i][np.newaxis, :], anchors_xywh)
                iou.append(iou_scale)
                iou_mask = iou_scale > 0.3
                
                #print("\n\t iou_mask =",len(iou_mask)

                if np.any(iou_mask):
                    xind, yind = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32)

                    label[i][yind, xind, iou_mask, :] = 0
                    label[i][yind, xind, iou_mask, 0:4] = bbox_xywh
                    label[i][yind, xind, iou_mask, 4:5] = 1.0
                    label[i][yind, xind, iou_mask, 5:6] = smooth_onehot # change
                    
                    #print("\n\t bbox_xywh:",bbox_xywh.shape)
                    #print("\n\t shape:",label[i][yind, xind, iou_mask, 6:].shape)
                    #self.tempText1
                    #print("\n\t key:",key)
                    tempVec=self.tempText1[key]
                    #print("\n\ttempVec.keys():",tempVec.keys())
                    
                    tempPhoc=tempVec['phoc']#.astype()
                    tempPhos=tempVec['phos']
                    #print("\n\t tempPhosc=",type(tempPhoc),"\t len:",len(tempPhoc))
                    #print("\n\t tempPhosc=",type(tempPhos),"\t len:",len(tempPhos))

                    #tempPhosc=tempPhoc+tempPhos
                    label[i][yind, xind, iou_mask, 6:610] = tempPhoc  # change
                    label[i][yind, xind, iou_mask, 610:] = tempPhos # change
                 
                    bbox_ind = int(bbox_count[i] % self.max_bbox_per_scale)
                    bboxes_xywh[i][bbox_ind, :4] = bbox_xywh
                    bbox_count[i] += 1

                    exist_positive = True

            if not exist_positive:
                best_anchor_ind = np.argmax(np.array(iou).reshape(-1), axis=-1)
                best_detect = int(best_anchor_ind / self.anchor_per_scale)
                best_anchor = int(best_anchor_ind % self.anchor_per_scale)
                xind, yind = np.floor(bbox_xywh_scaled[best_detect, 0:2]).astype(np.int32)

                label[best_detect][yind, xind, best_anchor, :] = 0
                label[best_detect][yind, xind, best_anchor, 0:4] = bbox_xywh
                label[best_detect][yind, xind, best_anchor, 4:5] = 1.0
                label[best_detect][yind, xind, best_anchor, 5:6] = smooth_onehot # change
                label[best_detect][yind, xind, best_anchor, 6:] = np.zeros(769) # change 

                bbox_ind = int(bbox_count[best_detect] % self.max_bbox_per_scale)
                bboxes_xywh[best_detect][bbox_ind, :4] = bbox_xywh
                bbox_count[best_detect] += 1

        if self.train_yolo_tiny:
            label_mbbox, label_lbbox = label
            mbboxes, lbboxes = bboxes_xywh
            return label_mbbox, label_lbbox, mbboxes, lbboxes

        label_sbbox, label_mbbox, label_lbbox = label
        sbboxes, mbboxes, lbboxes = bboxes_xywh
        return label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes

    def __len__(self):
        return self.num_batchs


'''
    search record in dataframe
'''

In [7]:
np.random.randint(1, 7,10)

array([2, 5, 5, 2, 4, 6, 1, 3, 3, 6])

In [9]:
'''

f = open("./data/tempCord.json")
tempCord1= json.load(f)

f = open("./data/tempText.json")
tempText1 = json.load(f)
'''
import json
trainset = Dataset1('train')
#testset = Dataset('test')

for batch_no,(image_data, target) in enumerate(trainset):
    #nm=str(np.random.randint(10000))+".png"
    print("\n\t image_data=",image_data.shape)    
    print("\n\t target:",target[0][0].shape)
    #input("image")
    #break


	 annotation name= /home/k/phd/yolov5/data/datasets/forms/l04-087.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/d01-024.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/n06-182.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/k04-022.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/c02-000.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/g06-026k.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/m02-112.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/a0


	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/c03-094c.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/p06-047.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/f04-074.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/a01-011.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/b04-103.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/a01-049u.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 775)

	 annotation name= /home/k/phd/yolov5/data/datasets/forms/g07-074a.png

	 image_data= (1, 416, 416, 3)

	 target: (1, 52, 52, 3, 7

'''
    dataset used here is forms data (IAM handwritten not sure) which is at times character z
    level so for this need to segment characters which forms a word
'''

In [ ]:
'[ 608 2287  694 2336]'
'[603, 603, 689, 652]'
'[77, 8, 78, 8]'
ss='[603 603 689 652]'.split(" ")
", ".join(ss)

In [ ]:
'[1763, 2064, 1849, 2113]'

In [ ]:
import json
f = open("./data/tempCord.json")
tempCord1= json.load(f)
o=[603, 603, 689, 652]

# tempCord1['[603, 603, 689, 652]']
# tempCord1['[77, 8, 78, 8]']


f = open("./data/tempText.json")
tempText1 = json.load(f)
tempText1
#['[1763,2064,1849,2113]']
#f = open("./data/tempText.json")
#tempText1 = json.load(f)

In [ ]:
'''
    In below for loop
    image_data: Actual Image
    
    image_data: (1, 416, 416, 3):(batch_size,height,width,channels) are returned feature maps from yolo
    target[0][0]: (1, 52, 52, 3, 6) here 3 indicates # anchors boxes
    target[1][0]: (1, 26, 26, 3, 6) here 3 indicates # anchors boxes
    target[2][0]: (1, 13, 13, 3, 6) here 3 indicates # anchors boxes
    
    where every feature map has dim 6 which is (midX,midY,height,width,objectness,classProb)

    target[0][1],target[1][1],target[2][1] is of shape (1, 100, 4)
    Where I think 100 is max object detected and 4 are midX,midY,height,width
    
'''

def shapes(image_data,target):

    print("\n\t image_data:",image_data.shape)
    print("\n\t target 0:",target[0][0].shape)
    print("\n\t target 1:",target[1][0].shape)
    print("\n\t target 2:",target[2][0].shape)
    print("\n\t target:",target[2][1].shape)
    print("\n\t target len:",len(target[0][0]))
    print("\n\t target len:",len(target))


In [ ]:

for batch_no,(image_data, target) in enumerate(trainset):
    if 0:
        shapes(image_data,target)
    
    break

'''
    Below part from PHOSC 
'''